In [1]:
import streamlit as st
import pandas as pd
import numpy as np 
#import streamlit_authenticator as stauth

import gspread
from google.oauth2.service_account import Credentials
import plotly.graph_objects as go
import plotly.express as px

In [2]:
%load_ext autoreload
%autoreload 2

import sys

if '../functions' in sys.path: 
    pass 
else: 
    sys.path.append('../functions')

#from functions_helpers import *
from functions_instruments import *

sys.path
%pwd

import ast
with open('secrets.txt') as f: 
    secrets_raw = f.read()

secrets = ast.literal_eval(secrets_raw)

#secrets

In [9]:
df_list = get_data_google_sheets([0,1], secrets)
tdf = generate_record_raw(df_list[0])

In [5]:
tdf.columns

Index(['발주처', '방영 채널', '작품명 / 시즌', 'Ep', 'RT', 'RTM', 'CPE', 'CPER', '환율',
       '영상 종류', '작업 종류', '마감일', '대금 수령일', '단가', '번역료', '수령액', '비고',
       'new_unit_price'],
      dtype='object', name=0)

tdf.columns

## Formula for new RTM

$RT \times RTM + CPE \times CPER = RTM2 \times RT$

$RTM2 = RMT + \dfrac{CPE \times CPER}{RT}$

In [23]:
df2 = summarise_by_v2(tdf, time_by='대금 수령일', period_by='All', group_by=[])
df2 = summarise_by_v2(tdf, time_by='대금 수령일', period_by='All', group_by=['발주처'])
df2 = summarise_by_v2(tdf, time_by='대금 수령일', period_by='All', group_by=['발주처', '방영 채널'])
#df2 = summarise_by_v2(tdf, time_by='대금 수령일', period_by='M', group_by=[])
tdf2 = df2.reset_index()
tdf2 = tdf2.groupby(['발주처'])['수령액'].sum().reset_index()
tdf2.columns = ['발주처', '총수령액']

tdf3 = pd.merge(df2.reset_index(), tdf2, on='발주처', how='left')
tdf3.sort_values(by='총수령액', ascending=False)

,발주처,방영 채널,단가 평균,번역료,수령액,총수령액
0,WhatSub,Disney+,3161.133603,32271300.0,31206366.0,50490430.0
7,WhatSub,Coupang,2192.857143,2337700.0,2260555.0,50490430.0
26,WhatSub,?,6000.000000,54000.0,52218.0,50490430.0
24,WhatSub,AXA,5000.000000,165000.0,159555.0,50490430.0
18,WhatSub,Wow Tech,4500.000000,279000.0,269794.0,50490430.0
16,WhatSub,NBCUniversal,1950.000000,469150.0,453668.0,50490430.0
12,WhatSub,Facebook,5000.000000,780000.0,754260.0,50490430.0
15,WhatSub,ESPN,5000.000000,510000.0,493170.0,50490430.0
29,WhatSub,Paramount,4500.000000,13500.0,13055.0,50490430.0
2,WhatSub,Netflix,3783.333333,6864800.0,6638265.0,50490430.0


In [10]:
'#extract_metrics(tdf)

(82080243.0,
 7272131.0,
 0.19620643631856013,
 2412677.0,
 5682763.0,
 -0.5754394473251832,
 '2022-09',
 6522851.0)

In [96]:
def gen_chart(df, period_by, group_by):
    df = df.reset_index()
    #df = df.sort_values(by='수령액', ascending=True)
    if   (group_by == []) & (period_by == 'All'):
        return None
    elif (len(group_by) == 1) & (period_by == 'All'):
        y_var = group_by[0]
        return px.bar(df, y=y_var, x="수령액", title=f"Total earned by {y_var}", color_discrete_sequence=px.colors.qualitative.Antique, orientation='h')
    elif (len(group_by) > 1) & (period_by == 'All'):
        y_var = group_by[0]
        color_var = group_by[1]
        return px.bar(df, y=y_var, x="수령액", color=color_var, title=f"Total earned by {y_var} and {color_var}", color_discrete_sequence=px.colors.qualitative.Antique, orientation='h')
    elif (group_by == []) & (period_by != 'All'):
         #return df
        return px.bar(df, y='time', x="수령액", title=f"Timely earned", color_discrete_sequence=px.colors.qualitative.Antique, orientation='h')
    else: 
        color_var = group_by[0]
        #cols = ['time', '수령액'] + [color_var]
        #df = df[cols].sort_values(by='수령액', ascending=True)
        fig = px.bar(df, y='time', x="수령액", color=color_var, title=f"Timely earned by {color_var}", color_discrete_sequence=px.colors.qualitative.Antique, orientation='h')
        return fig.update_layout(barmode='stack')

In [12]:
period_by, group_by = 'M', ['발주처', '작업 종류', '영상 종류']
#period_by, group_by = 'Y', []
period_by, group_by = 'All', []
df2 = summarise_by_v2(tdf, time_by='대금 수령일', period_by=period_by, group_by=group_by)
df2.reset_index()
gen_chart(df2, period_by=period_by, group_by=group_by)